In [14]:

import numpy as np
import pandas as pd


In [15]:
import os
import pandas as pd
import numpy as np

os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'


In [16]:
calendar = pd.read_csv('./data/calendar.csv')
train_val = pd.read_csv('./data/sales_train_validation.csv')


In [17]:
cal_val = calendar.copy()

for i in range(1914, 1969):
    cal_val.drop(index=i, axis=0, inplace = True)
cal_val = cal_val[['date', 'weekday', 'd']]
cal_val['d'] = cal_val['d'].str.split('_').str[1]
cal_val

train_val['id'] = train_val['store_id'] + str('_') + train_val['dept_id']
train_val = train_val.drop(columns = ['store_id', 'dept_id', 'item_id', 'cat_id', 'state_id'], axis=1)

for col in train_val.columns[1:]:
    train_val.rename(columns = {col: col.split('_')[-1]}, inplace=True)

train_group = train_val.groupby('id')
train_group = train_group.sum()
train_group.reset_index(inplace=True)
train_group

train_melt = pd.melt(train_group, id_vars = 'id', 
                  value_vars = [col for col in train_group.columns if not col.startswith('i')],
                  var_name = 'd', value_name = 'sales')

train_melt = train_melt.merge(cal_val, on ='d', copy=False)

train_melt['d'] = train_melt['d'].astype('int64')
train_melt['date'] = pd.to_datetime(train_melt['date'])
train_melt = train_melt.sort_values(by=['id', 'd'], ascending=True)

train_melt.reset_index(inplace=True)
train_melt.drop(columns='index',inplace=True)

In [18]:
train_melt

,id,d,sales,date,weekday
0,CA_1_FOODS_1,1,297,2011-01-29,Saturday
1,CA_1_FOODS_1,2,284,2011-01-30,Sunday
2,CA_1_FOODS_1,3,214,2011-01-31,Monday
3,CA_1_FOODS_1,4,175,2011-02-01,Tuesday
4,CA_1_FOODS_1,5,182,2011-02-02,Wednesday
...,...,...,...,...,...
133905,WI_3_HOUSEHOLD_2,1909,148,2016-04-20,Wednesday
133906,WI_3_HOUSEHOLD_2,1910,142,2016-04-21,Thursday
133907,WI_3_HOUSEHOLD_2,1911,166,2016-04-22,Friday
133908,WI_3_HOUSEHOLD_2,1912,232,2016-04-23,Saturday


In [19]:
train_melt.to_pickle("./data/m5_preprocessed.pkl")

In [20]:
df = pd.read_pickle('./data/m5_preprocessed.pkl')

In [49]:
def preprocess(data, num_samples, ):
    data_dict = {}
    data_len = 30
    jump = 1
    input_size = 1
    output_size = 1
    
    id_col='id'
    input_cols = 'sales'
    target_col = 'sales'
    time_col = 'd'
    
    sampling_end_locations = []
    split_data_map = {}
    
    for id_name, df in data.groupby(id_col):
        num_entries = len(df)
        sampling_end_locations += [
            (id_name, data_len + i)
            for i in range(num_entries - data_len  + 1)
            if (i % jump) == 0
        ]
        
        split_data_map[id_name] = df
        
    if num_samples == 'max':
        samples = len(sampling_end_locations)

    else: 
        samples = num_samples

    inputs = np.zeros((samples, data_len, input_size))
    outputs = np.zeros((samples, data_len, output_size))
    time = np.empty((samples, data_len, 1), dtype=object)
    identifiers = np.empty((samples, data_len+ , 1), dtype=object)

    if (samples > 0) & (len(sampling_end_locations) >= samples):
        print('Extracting {} smaples...'.format(samples))

        ranges = [
            sampling_end_locations[i] for i in np.random.choice(
                len(sampling_end_locations), samples, replace = True)
        ]

    else:
        raise ValueError('Smaple is over')

    for i, tup in enumerate(ranges):
        if((i+1)% 1000) == 0:
            print(i+1, 'of', smaples, 'smaples done...')

        id_name, end_idx = tup
        sliced = split_data_map[id_name].iloc[end_idx-data_len : end_idx]

        outputs[i, :, :] = sliced[[target_col]]
        inputs[i, :, :] = sliced[[input_cols]]
        time[i, :, 0] = sliced[time_col]
        identifiers[i, :, 0] = sliced[id_col]

    data_dict['inputs'] = inputs
    data_dict['targets'] = outputs
    data_dict['time'] = time
    data_dict['identifier'] = identifiers

    return data_dict

In [52]:
data_dict= preprocess(df, 100)

Extracting 100 smaples...
